# Lbel Conversion
This file contain code for the proper converstaion of the labelme file to a standardized format.

In [ ]:
import os

# Standard Libraries
import math
from PIL import Image 
from ultralytics import YOLO
import xml.etree.ElementTree as ET
import pandas as pd

from textwrap import wrap
import cv2
import matplotlib.pyplot as plt

### convert labelme to the YOLO format - testing on one file
The output format from the model is [class] [x_center] [y_center] [width] [height] [confidence]. 

This is the result from the model argument of the predict function. In the function, I used save_txt = True, and not saving the result in a separate line. The result format is in the option above.


The following function was used for yolo results:
``` python
results = model.predict(source=imagePath, device = "cuda:0", agnostic_nms = True, save = False, save_txt = True, save_conf = True, conf = 0.75, visualize = False, batch = 32, project = "../ssic_image-corpus/scripts/baseline/batchAnother")
```

In [80]:
def convert_annotations_to_yolo(input_folder, output_folder, face_class_id=99):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.xml'):
            file_path = os.path.join(input_folder, file_name)
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Get image dimensions
            image_width = int(root.find("imagesize/ncols").text)
            image_height = int(root.find("imagesize/nrows").text)

            # Prepare to collect YOLO data
            yolo_data = []

            # Iterate through each object
            for obj in root.findall("object"):
                class_name = obj.find("name").text  # Class name
                class_id = face_class_id if class_name.upper() == "FACE" else -1  # Mapping FACE to specified class ID

                # Collect polygon points
                polygon = obj.find("polygon")
                x_min = float(polygon.find("pt[1]/x").text)
                y_min = float(polygon.find("pt[1]/y").text)
                x_max = float(polygon.find("pt[3]/x").text)
                y_max = float(polygon.find("pt[3]/y").text)

                # Calculate YOLO parameters
                x_center = round(((x_min + x_max) / 2) / image_width, 5)
                y_center = round(((y_min + y_max) / 2) / image_height, 5)
                width = round((x_max - x_min) / image_width, 5)
                height = round((y_max - y_min) / image_height, 5)
                confidence = 1.0  # Default confidence for conversion purposes

                # Append data
                yolo_data.append([class_id, x_center, y_center, width, height, confidence])

            # Convert to a DataFrame
            columns = ["class", "x_center", "y_center", "width", "height", "confidence"]
            yolo_df = pd.DataFrame(yolo_data, columns=columns)

            # Save the data to a YOLO annotation file format
            base_name = os.path.splitext(file_name)[0]
            output_file_path = os.path.join(output_folder, f"{base_name}.txt")
            yolo_df.to_csv(output_file_path, sep=" ", header=False, index=False)

    print(f"Converted YOLO annotations saved in: {output_folder}")


In [ ]:
convert_annotations_to_yolo("../ssic_image-corpus/data_phase-4_consensus/face/label_only", 
                            "../ssic_image-corpus/data_phase-4_consensus/face/label_only_yolo")